In [14]:
import os, shutil, datetime

In [15]:
def move_files(src_folder, dest_folder):
    if os.path.exists(src_folder):
        for root, _, files in os.walk(src_folder):
            for file in files:
                src_path = os.path.join(root, file)
                dest_path = os.path.join(dest_folder, file)

                # 파일명이 중복되면 '_1', '_2' 등 숫자를 추가하여 저장
                if os.path.exists(dest_path):
                    base, ext = os.path.splitext(file)
                    counter = 1
                    while os.path.exists(dest_path):
                        new_file_name = f"{base}_{counter}{ext}"
                        dest_path = os.path.join(dest_folder, new_file_name)
                        counter += 1

                shutil.move(src_path, dest_path)

In [16]:
def create_folders(base_folder, folders):
    for folder in folders:
        path = os.path.join(base_folder, folder)
        os.makedirs(path, exist_ok=True)

In [18]:
def remove_processed_files(base_directory):
    """ 전체 폴더에서 .processed 파일 삭제 """
    for root, _, files in os.walk(base_directory):
        for file in files:
            if file == ".processed":
                os.remove(os.path.join(root, file))
                print(f"Removed: {os.path.join(root, file)}")

In [ ]:
def organize_product_folder(productname_folder):
    processed_file = os.path.join(productname_folder, ".processed")
    
    # 이미 정리된 폴더인지 확인
    if os.path.exists(processed_file):
        print(f"Skipping already processed folder & Rename_files : {productname_folder}")
        remove_processed_files(productname_folder)
        return
    
    original_product = os.path.join(productname_folder, 'OriginalProduct')
    sample_folder = os.path.join(productname_folder, 'Sample')
    
    # Sample 및 OriginalProduct가 존재할 경우
    if os.path.exists(sample_folder) and os.path.exists(original_product):
        move_files(sample_folder, productname_folder)
        shutil.rmtree(sample_folder)
        shutil.move(os.path.join(original_product, 'BasicAsset'), productname_folder)
        shutil.move(os.path.join(original_product, 'SourceFile'), productname_folder)
        shutil.move(os.path.join(original_product, 'EditedContent'), productname_folder)
        shutil.rmtree(original_product)
    
    # BasicAsset 폴더 정리 및 새 폴더 생성
    basic_asset = os.path.join(productname_folder, 'BasicAsset')
    create_folders(basic_asset, ['Logo', '2D', '3D', 'Cutout'])

    cutout_folder = os.path.join(basic_asset, 'Cutout')
    create_folders(cutout_folder, ['Product', 'Texture', 'Sample'])
    
    # 기존 폴더 내 파일 정리
    move_files(os.path.join(basic_asset, 'Prod_Cutout'), os.path.join(cutout_folder, 'Product'))
    move_files(os.path.join(basic_asset, 'Texture_Cutout'), os.path.join(cutout_folder, 'Texture'))
    shutil.rmtree(os.path.join(basic_asset, 'Prod_Cutout'))
    shutil.rmtree(os.path.join(basic_asset, 'Texture_Cutout'))
    shutil.rmtree(os.path.join(basic_asset, 'Prod_Logo'))
    shutil.rmtree(os.path.join(basic_asset, 'Prod_2D'))
    shutil.rmtree(os.path.join(basic_asset, 'Prod_3D'))
    
    for entry in os.scandir(productname_folder):
        if entry.is_file():
            shutil.move(entry.path, os.path.join(cutout_folder, 'Sample'))
    # move_files(productname_folder, os.path.join(cutout_folder, 'Sample'))

    # 정리 완료 후 체크포인트 파일 생성
    with open(processed_file, 'w') as f:
        f.write("processed")
    
    

In [20]:
def mark_existing_processed_folders(base_directory):
    """ 기존에 정리된 폴더에도 .processed 파일을 생성 """
    for brand_line in os.listdir(base_directory):
        brand_line_path = os.path.join(base_directory, brand_line)

        # "0_BrandAsset" 폴더는 건너뛰기
        if brand_line == "0_BrandAsset":
            print(f"Skipping brand asset folder: {brand_line}")
            continue

        # BrandLine 폴더 내 BrandName 폴더 찾기
        if os.path.isdir(brand_line_path):
            for product_name in os.listdir(brand_line_path):
                product_name_path = os.path.join(brand_line_path, product_name)
                print(f"Processing: {product_name}")    
                processed_file = os.path.join(product_name_path, ".processed")
                basic_asset = os.path.join(product_name_path, 'BasicAsset')
                
                # 이미 .processed 파일이 있으면 건너뜀
                if os.path.exists(processed_file):
                    continue

                # BasicAsset/Cutout 구조가 존재하면 정리된 것으로 간주하고 .processed 생성
                if os.path.exists(os.path.join(basic_asset, 'Cutout')) :
                    print(f"Marking existing processed folder: {product_name_path}")
                    with open(processed_file, 'w') as f:
                        f.write("processed")

In [21]:
def process_all_product_folders(base_directory):
    for brand_line in os.listdir(base_directory):
        brand_line_path = os.path.join(base_directory, brand_line)

        # "0_BrandAsset" 폴더는 건너뛰기
        if brand_line == "0_BrandAsset":
            print(f"Skipping brand asset folder: {brand_line}")
            continue

        # BrandLine 폴더 내 BrandName 폴더 찾기
        if os.path.isdir(brand_line_path):
            for product_name in os.listdir(brand_line_path):
                product_name_path = os.path.join(brand_line_path, product_name)
                print(f"Processing: {product_name}")
                organize_product_folder(product_name_path)

In [22]:
# 이미 존재했던 폴더에 대해서도 파일명 변경
base_directory = "C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중"  # 실제 경로로 변경하세요
mark_existing_processed_folders(base_directory)
process_all_product_folders(base_directory)
remove_processed_files(base_directory)

Skipping brand asset folder: 0_BrandAsset
Processing: BarrierEmulsion
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\BarrierEmulsion
Processing: BarrierToner
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\BarrierToner
Processing: Bomb
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\Bomb
Processing: EyeSerum
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\EyeSerum
Processing: VCream
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\VCream
Processing: VCreamAD
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\VCreamAD
Processing: VEyeSerum
Marking existing processed folder: C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중\AgeKnockdown\VEyeSerum
Processing: WaterEssence
Marking existing processe